<a href="https://colab.research.google.com/github/levulinh/COMap/blob/master/f2s_gd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizer for searching for W, h and l



## Fix esp<sub>re</sub> to find W and h

### Import necessary libraries

In [3]:
# Import libraries

import autograd.numpy as np
from autograd import grad
import random
from typing import Set, Tuple, List
import csv

### CONSTANTS

In [4]:
# CONSTANTS
eps = 1e-3
eta = 0.001
re0 = 10.
re1 = 25.

u_0 = 4*np.pi*1e-7*1e-6
esp_0 = 8.8*1e-12*1e-6

inf = 999999

# Look up table for values of h by top and bottom layer material
h_arr = [[inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf],   # M1
         [0.175, inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf],   # M2
         [0.570, 0.175, inf,   inf,   inf,   inf,   inf,   inf,   inf],   # M3
         [0.965, 0.570, 0.175, inf,   inf,   inf,   inf,   inf,   inf],   # M4
         [1.360, 0.965, 0.570, 0.175, inf,   inf,   inf,   inf,   inf],   # M5
         [1.175, 1.360, 0.965, 0.570, 0.175, inf,   inf,   inf,   inf],   # M6
         [2.150, 1.175, 1.360, 0.965, 0.570, 0.175, inf,   inf,   inf],   # M7
         [2.965, 2.570, 2.175, 1.780, 1.385, 0.990, 0.595, inf,   inf],   # M8
         [4.605, 4.210, 3.815, 3.420, 3.025, 2.630, 2.235, 0.740, inf]]   # M9        

# Look up table for values of t by top layer
t_arr = [0.18, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.9, 3.4]

# Look up table for minimum value of W by top layer
min_W = [0.09, 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.4, 2. ]

### Define the model for auto-grad

In this part, I use gradient descent to minimize the quadratic loss function.
By squaring up the "loss" (difference of Z<sub>0</sub> and $\bar{Z_{0}}$ = 50).<br/>
See SWTL-Calculation.docx for reference.

#### Using First equation (with re is assumed)

In [5]:
# Define the functions for optimizing process

# Forward function
def Z0(W, h, re):
    return (120*np.pi)/(np.sqrt(re)*(W/h + 1.393 + 0.667*np.log(W/h + 1.444)))

# Cost function
def Loss(W, h, re):
    return (Z0(W, h, re) - 50)**2

# Auto Gradient function
def grad_L(W, h, re):
    gd = grad(Loss, (0, 1))
    gd_val = gd(W, h, re)
    return gd_val

def closest_to_h_arr(h):
    A = np.array(h_arr)
    X = np.abs(A - h)
    idx = np.where(X == X.min())
    top = idx[0][-1]
    bottom = idx[1][-1]
    return A[np.where(X == X.min())][0], (top, bottom)


def update_Wh(W, h, grad_val):
    new_h, (top, bottom) = closest_to_h_arr(h - eta * grad_val[1])
    new_w = W - eta * grad_val[0]
    return new_w, new_h, top, bottom

def train(W, h, re, max_iter=1000, debug=True):
    iter = 0
    top = 0
    bottom = 0
    L_train = inf # infinity

    while (iter<max_iter) and (L_train > eps):
        # Forward
        gd_val = grad_L(W, h, re)
        # print(gd_val)

        # Backward update
        W, h, top, bottom = update_Wh(W, h, gd_val)
        L_train = Loss(W, h, re)
        #print(h)

        if debug:
            print("iter: {}     W: {}, h: {}, L: {}". format(iter, W, h, L_train))

        # Next iter
        iter += 1
    print("iter: {}     W: {:.3f}, h: {}, L: {:.2e}". format(iter, W, h, L_train))
    print("top layer: M{}, bottom layer: M{}". format(top + 1, bottom + 1))

    return W, h, top, bottom

#### Using Second equation (find l with found W and h)

In [6]:
"""
    Optimization for finding l
"""

def L(W, h):
    return (u_0/(4*np.pi)) * np.log(1 + (32*h**2/W**2) * (1 + np.sqrt(((np.pi*W)/(8*h))**2 + 1)))

def C_plate(W, h, l, top, bottom):
    esp_r = 3.4
    if top==8 and bottom==7:
        esp_r = 4.3

    return esp_0*esp_r*W*l/h

def C_fringe(W, h, l, top, bottom):
    n = 80
    esp_r = 3.4
    t = t_arr[top]

    if top==8 and bottom==7:
        esp_r = 4.3
    
    C_0 = (2*esp_0*esp_r/np.pi) * W*l
    C_1 = sum(1/((i-1)*t + n*h) for i in range(1, n+1))
    return C_0 * C_1

def C(W, h, l, top, bottom):
    return C_plate(W, h, l, top, bottom) + 2 * C_fringe(W, h, l, top, bottom)

def Z0_1(W, h, l, top, bottom):
    return np.sqrt(L(W, h) / C(W, h, l, top, bottom))

def Loss_1(W, h, l, top, bottom):
    return (Z0_1(W, h, l, top, bottom) - 50)**2

def grad_L_1(W, h, l, top, bottom):
    gd = grad(Loss_1, 2)
    gd_val = gd(W, h, l, top, bottom)
    return gd_val

def pseudo_grad(W, h, l, top, bottom):
    z0_p = Loss_1(W, h, l-eps, top, bottom)
    z0_n = Loss_1(W, h, l+eps, top, bottom)
    return (z0_n - z0_p)/(2*eps)

def train_1(W, h, l, top, bottom, max_iter=1000, debug=True):
    iter = 0
    L_train = inf # infinity

    while iter < max_iter and L_train > eps:
        # Forward
        gd_val = grad_L_1(W, h, l, top, bottom)

        # Backward Update
        l = l - eta * gd_val
        L_train = Loss_1(W, h, l, top, bottom)

        if debug:
            print("iter: {}     l: {}, L: {}". format(iter, l, L_train))

        # Next iter
        iter += 1
    print("iter: {}     l: {:.2f}, L: {:.2e}". format(iter, l, L_train))
    return l

### Run the optimizer

In [7]:
#####
# Initial value for root scanner
#####

re = 10
W_0 = 6.
h_0 = 3.
l = 1.

Grid search function to sweep all possible roots<br>
- W_grid: number of intervals should the function divide W into<br>
- h_grid is not needed<br>
- return: List of (W, h, top, bottom, l) tuples

In [8]:
# Supporting function for splitting
def split_equal(min, max, parts):
    min = float(min)
    max = float(max)
    return [min + i*(max-min)/parts for i in range(0,parts+1)]

h_set : Set[float] = set()
results : List[Tuple[float, float, int, int, float]] = []

for i in range(0, 9):
    for j in range(0, 9):
        if h_arr[i][j] != inf:
            h_set.add(h_arr[i][j])

def grid_search(W_grid):
    for W in split_equal(2, 12, W_grid):
        for h in list(h_set):
            print("START FOR W={} and h={} ------------------". format(W, h))
            l = 6.
            W_r, h_r, top, bottom = train(W, h, re, debug=False)
            if W_r < min_W[top] or W_r > 12.:
                print("No l found!")
                continue
            l = train_1(W_r, h_r, l, top, bottom, debug=False)
            if l <= 12.:
                results.append((W_r, h_r, top+1, bottom+1, l))

Execution

In [9]:
grid_search(20)
print("FINISHED !!!")

START FOR W=2.0 and h=0.175 ------------------
iter: 25     W: 1.829, h: 3.42, L: 7.72e-04
top layer: M9, bottom layer: M4
No l found!
START FOR W=2.0 and h=0.57 ------------------
iter: 16     W: 1.200, h: 2.235, L: 7.31e-04
top layer: M9, bottom layer: M7
No l found!
START FOR W=2.0 and h=1.175 ------------------
iter: 12     W: 0.955, h: 1.78, L: 7.81e-04
top layer: M8, bottom layer: M4
iter: 144     l: 6.63, L: 9.51e-04
START FOR W=2.0 and h=0.965 ------------------
iter: 12     W: 0.955, h: 1.78, L: 5.95e-04
top layer: M8, bottom layer: M4
iter: 144     l: 6.63, L: 9.60e-04
START FOR W=2.0 and h=1.36 ------------------
iter: 12     W: 0.956, h: 1.78, L: 9.97e-04
top layer: M8, bottom layer: M4
iter: 143     l: 6.63, L: 9.97e-04
START FOR W=2.0 and h=2.15 ------------------
iter: 18     W: 1.200, h: 2.235, L: 9.23e-04
top layer: M9, bottom layer: M7
No l found!
START FOR W=2.0 and h=2.965 ------------------
iter: 27     W: 1.624, h: 3.025, L: 8.02e-04
top layer: M9, bottom layer: M

In [10]:
with open("/content/drive/My Drive/QLO_outputs/result1_{}.csv". format(re), "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["W", "h", "top", "bottom", "l"])
    writer.writerows(results)